In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.5 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from scipy import stats

In [4]:
transaction = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/customer_transaction_data.csv", parse_dates=['date'])
item = pd.read_csv("/content/drive/MyDrive/Project Data Mining/Predicting Coupon Redemption/item_data.csv")

In [5]:
transaction_item = transaction.merge(item, on='item_id', how='left', validate='many_to_one')
print(transaction_item)
print(transaction_item.info())
print(transaction_item.nunique())

              date  customer_id  item_id  quantity  selling_price  \
0       2012-01-02         1501    26830         1          35.26   
1       2012-01-02         1501    54253         1          53.43   
2       2012-01-02         1501    31962         1         106.50   
3       2012-01-02         1501    33647         1          67.32   
4       2012-01-02         1501    48199         1          71.24   
...            ...          ...      ...       ...            ...   
1324561 2013-06-30         1129     2777         1         284.60   
1324562 2013-06-30         1129     2953         4          42.74   
1324563 2013-06-30         1129     2971         6          64.12   
1324564 2013-06-30         1129    46984         1          95.82   
1324565 2013-06-30         1129    64498         2         489.78   

         other_discount  coupon_discount  brand   brand_type          category  
0                -10.69              0.0     56        Local  Natural Products  
1        

In [6]:
fp_brand = transaction_item.groupby(['date', 'customer_id'])['brand'].unique().reset_index(drop=True)

In [7]:
fp_brand

0                                                 [487, 56]
1         [56, 2320, 4467, 676, 115, 602, 1075, 1262, 97...
2         [984, 2098, 93, 56, 1709, 912, 115, 686, 124, ...
3         [1196, 1193, 56, 3138, 823, 984, 3194, 133, 19...
4         [2088, 1162, 1235, 673, 56, 89, 1088, 5309, 2074]
                                ...                        
142440    [360, 4991, 1013, 56, 1124, 602, 278, 1558, 11...
142441             [4467, 4890, 1401, 1444, 89, 3521, 1180]
142442                                       [56, 278, 172]
142443                            [461, 56, 1100, 910, 522]
142444    [1000, 255, 544, 946, 1636, 4700, 1105, 1494, ...
Name: brand, Length: 142445, dtype: object

In [8]:
fp_cate = transaction_item.groupby(['date', 'customer_id'])['category'].unique().reset_index(drop=True)

In [9]:
fp_cate

0                                                 [Grocery]
1         [Grocery, Pharmaceutical, Natural Products, Ba...
2         [Packaged Meat, Pharmaceutical, Grocery, Natur...
3         [Pharmaceutical, Grocery, Seafood, Prepared Fo...
4                                 [Pharmaceutical, Grocery]
                                ...                        
142440    [Grocery, Dairy, Juices & Snacks, Natural Prod...
142441                      [Pharmaceutical, Grocery, Meat]
142442                                            [Grocery]
142443               [Grocery, Bakery, Packaged Meat, Fuel]
142444              [Packaged Meat, Grocery, Prepared Food]
Name: category, Length: 142445, dtype: object

In [10]:
fp_item = transaction.groupby(['date', 'customer_id'])['item_id'].unique().reset_index(drop=True)

In [11]:
fp_item

0                              [10537, 19560, 21653, 32083]
1         [4642, 5566, 9309, 9645, 12433, 12543, 13174, ...
2         [5525, 6497, 6588, 7005, 7364, 8145, 8261, 979...
3         [4408, 5454, 5797, 5865, 7200, 9198, 9532, 980...
4         [10423, 10565, 17035, 27801, 30645, 30832, 340...
                                ...                        
142440    [6084, 11115, 11350, 11799, 12031, 13174, 1421...
142441    [7898, 9724, 10258, 11354, 24257, 27787, 28089...
142442                          [8245, 12433, 27309, 34535]
142443    [9760, 10789, 18441, 21332, 25239, 29423, 3217...
142444    [6364, 8585, 9235, 9429, 12521, 14984, 30924, ...
Name: item_id, Length: 142445, dtype: object

In [12]:
N = len(fp_item)

In [13]:
te = TransactionEncoder()
te_array = te.fit(fp_item).transform(fp_item)
df = pd.DataFrame(te_array, columns=te.columns_)

In [14]:
min_support = 0.05
d = fpgrowth(df, min_support=min_support, use_colnames=True)
d

,support,itemsets
0,0.093067,(49009)


In [15]:
item[item.item_id==49009]

,item_id,brand,brand_type,category
49008,49009,56,Local,Fuel


In [ ]:
min_support = 0.03
d2 = fpgrowth(df, min_support=min_support, use_colnames=True)
d2

,support,itemsets
0,0.043252,(34047)
1,0.093067,(49009)


In [ ]:
item[item.item_id==34047]

,item_id,brand,brand_type,category
34046,34047,56,Local,Grocery


In [ ]:
min_support = 0.01
d3 = fpgrowth(df, min_support=min_support, use_colnames=True)
d3

,support,itemsets
0,0.027119,(13174)
1,0.010544,(30518)
2,0.022921,(45502)
3,0.012166,(53517)
4,0.043252,(34047)
5,0.013991,(9281)
6,0.010439,(53521)
7,0.012222,(5798)
8,0.014707,(32991)
9,0.013058,(6958)


In [ ]:
min_support = 0.005
d4 = fpgrowth(df, min_support=min_support, use_colnames=True) 
d4

,support,itemsets
0,0.027119,(13174)
1,0.010544,(30518)
2,0.008726,(14172)
3,0.007603,(23417)
4,0.022921,(45502)
...,...,...
76,0.015515,(18156)
77,0.005651,(48973)
78,0.007477,(15874)
79,0.007287,(27608)


In [ ]:
(d4.groupby('itemsets').apply(len) > 1).sum()

0

In [ ]:
min_support = 0.001
d5 = fpgrowth(df, min_support=min_support, use_colnames=True) 
d5

,support,itemsets
0,0.001987,(10537)
1,0.001229,(21653)
2,0.027119,(13174)
3,0.004690,(12433)
4,0.004002,(27435)
...,...,...
1659,0.001053,"(18156, 45502)"
1660,0.002036,"(18156, 34047)"
1661,0.001018,"(15874, 34047)"
1662,0.001130,"(27608, 34047)"


In [ ]:
d5[d5.itemsets.apply(len) > 1]

,support,itemsets
1598,0.002401,"(13174, 34047)"
1599,0.001292,"(49009, 13174)"
1600,0.001200,"(23417, 34047)"
1601,0.001229,"(13174, 45502)"
1602,0.002864,"(45502, 34047)"
...,...,...
1659,0.001053,"(18156, 45502)"
1660,0.002036,"(18156, 34047)"
1661,0.001018,"(15874, 34047)"
1662,0.001130,"(27608, 34047)"


In [ ]:
ar = association_rules(d5, metric='confidence', min_threshold=0.25)
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(28289),(25377),0.006087,0.008172,0.001973,0.324106,39.662625,0.001923,1.467432
1,(27801),(4649),0.003945,0.005307,0.001186,0.300712,56.659900,0.001165,1.422436
2,(11889),(27801),0.004240,0.003945,0.001165,0.274834,69.659771,0.001149,1.373555
3,(27801),(11889),0.003945,0.004240,0.001165,0.295374,69.659771,0.001149,1.413174
4,(46536),(46201),0.004051,0.003096,0.001257,0.310225,100.204180,0.001244,1.445260
5,(46201),(46536),0.003096,0.004051,0.001257,0.405896,100.204180,0.001244,1.676388
6,(53521),(53517),0.010439,0.012166,0.003412,0.326833,26.864202,0.003285,1.467442
7,(53517),(53521),0.012166,0.010439,0.003412,0.280439,26.864202,0.003285,1.375228
8,(53511),(53517),0.003629,0.012166,0.001530,0.421663,34.658886,0.001486,1.708061
9,(53511),(53521),0.003629,0.010439,0.001257,0.346228,33.166430,0.001219,1.513618


In [ ]:
def chi2_pvalue(x):
  return 1 - stats.chi2.cdf(x, 1)

def chi_square(df:pd.DataFrame, transaction_num:int):
  '''
  Input the association rules DataFrame and the number of total transactions.
  Return a DataFrame with three new columns indicating the chi square statistic, corresponding p-value (dof=1).
  leverage(A->B) = support(A->B) - support(A)*support(B), range: [-1, 1].
  '''
  x1 = df["antecedent support"]
  x2 = df["consequent support"]
  x3 = df["support"]
  df['chi_square'] = np.round(transaction_num * ((x3-x1*x2)**2/(x1*x2) + (x1-x3-x1*(1-x2))**2/(x1*(1-x2)) + (x2-x3-x2*(1-x1))**2/(x2*(1-x1)) + (1-x1-x2+x3-(1-x1)*(1-x2))**2/((1-x1)*(1-x2))), decimals=1)
  df['p_value'] = np.round(df['chi_square'].apply(chi2_pvalue), decimals=3)
  df['leverage'] = x3 - x2*x1
  return df

In [ ]:
def confidences(df:pd.DataFrame):
  '''
  Input the association rules DataFrame.
  Return a DataFrame with three new columns of confidences.
  '''
  x1 = df["antecedent support"]
  x2 = df["consequent support"]
  x3 = df["support"]
  df['confidence'] = x3/x1
  df['confidence_inverse'] = x3/x2
  df['lift'] = x3/(x1*x2)
  df['max_confidence'] = x3/np.minimum(x1, x2)
  df['all_confidence'] = x3/np.maximum(x1, x2)
  df['kulczynski'] = (x3/x1 + x3/x2)/2
  df['cosine'] = x3/np.sqrt(x1*x2)
  return df

In [ ]:
def imbalance_ratio(df:pd.DataFrame):
  x1 = df["antecedent support"]
  x2 = df["consequent support"]
  x3 = df["support"]
  df['imbalance_ratio'] = np.abs(x1 - x2)/(x1 + x2 - x3)
  return df

In [ ]:
chi_square(ar, transaction_num=N)
confidences(ar)
imbalance_ratio(ar)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,chi_square,p_value,confidence_inverse,max_confidence,all_confidence,kulczynski,cosine,imbalance_ratio
0,(28289),(25377),0.006087,0.008172,0.001973,0.324106,39.662625,0.001923,1.467432,10742.9,0.0,0.241409,0.324106,0.241409,0.282758,0.279718,0.169714
1,(27801),(4649),0.003945,0.005307,0.001186,0.300712,56.659900,0.001165,1.422436,9326.6,0.0,0.223545,0.300712,0.223545,0.262128,0.259273,0.168842
2,(11889),(27801),0.004240,0.003945,0.001165,0.274834,69.659771,0.001149,1.373555,11326.4,0.0,0.295374,0.295374,0.274834,0.285104,0.284919,0.042000
3,(27801),(11889),0.003945,0.004240,0.001165,0.295374,69.659771,0.001149,1.413174,11326.4,0.0,0.274834,0.295374,0.274834,0.285104,0.284919,0.042000
4,(46536),(46201),0.004051,0.003096,0.001257,0.310225,100.204180,0.001244,1.445260,17706.7,0.0,0.405896,0.405896,0.310225,0.358060,0.354851,0.162098
5,(46201),(46536),0.003096,0.004051,0.001257,0.405896,100.204180,0.001244,1.676388,17706.7,0.0,0.310225,0.405896,0.310225,0.358060,0.354851,0.162098
6,(53521),(53517),0.010439,0.012166,0.003412,0.326833,26.864202,0.003285,1.467442,12380.4,0.0,0.280439,0.326833,0.280439,0.303636,0.302748,0.089978
7,(53517),(53521),0.012166,0.010439,0.003412,0.280439,26.864202,0.003285,1.375228,12380.4,0.0,0.326833,0.326833,0.280439,0.303636,0.302748,0.089978
8,(53511),(53517),0.003629,0.012166,0.001530,0.421663,34.658886,0.001486,1.708061,7240.0,0.0,0.125793,0.421663,0.125793,0.273728,0.230310,0.598425
9,(53511),(53521),0.003629,0.010439,0.001257,0.346228,33.166430,0.001219,1.513618,5663.7,0.0,0.120377,0.346228,0.120377,0.233302,0.204151,0.531507


In [ ]:
ar = ar[ar.all_confidence>=0.25]
num = len(ar)

In [ ]:
ar.index=range(0, num)
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,chi_square,p_value,confidence_inverse,max_confidence,all_confidence,kulczynski,cosine,imbalance_ratio
0,(11889),(27801),0.004240,0.003945,0.001165,0.274834,69.659771,0.001149,1.373555,11326.4,0.0,0.295374,0.295374,0.274834,0.285104,0.284919,0.042000
1,(27801),(11889),0.003945,0.004240,0.001165,0.295374,69.659771,0.001149,1.413174,11326.4,0.0,0.274834,0.295374,0.274834,0.285104,0.284919,0.042000
2,(46536),(46201),0.004051,0.003096,0.001257,0.310225,100.204180,0.001244,1.445260,17706.7,0.0,0.405896,0.405896,0.310225,0.358060,0.354851,0.162098
3,(46201),(46536),0.003096,0.004051,0.001257,0.405896,100.204180,0.001244,1.676388,17706.7,0.0,0.310225,0.405896,0.310225,0.358060,0.354851,0.162098
4,(53521),(53517),0.010439,0.012166,0.003412,0.326833,26.864202,0.003285,1.467442,12380.4,0.0,0.280439,0.326833,0.280439,0.303636,0.302748,0.089978
5,(53517),(53521),0.012166,0.010439,0.003412,0.280439,26.864202,0.003285,1.375228,12380.4,0.0,0.326833,0.326833,0.280439,0.303636,0.302748,0.089978
6,(46201),(46436),0.003096,0.002927,0.001144,0.369615,126.258367,0.001135,1.581687,20378.0,0.0,0.390887,0.390887,0.369615,0.380251,0.380102,0.034532
7,(46436),(46201),0.002927,0.003096,0.001144,0.390887,126.258367,0.001135,1.636650,20378.0,0.0,0.369615,0.390887,0.369615,0.380251,0.380102,0.034532
8,(46282),(46436),0.003019,0.002927,0.001109,0.367442,125.516201,0.001100,1.576254,19633.4,0.0,0.378897,0.378897,0.367442,0.373169,0.373125,0.018868
9,(46436),(46282),0.002927,0.003019,0.001109,0.378897,125.516201,0.001100,1.605178,19633.4,0.0,0.367442,0.378897,0.367442,0.373169,0.373125,0.018868


In [ ]:
def drop_duplicate_association(df:pd.DataFrame):
  df['association'] = [ tuple(sorted(list(x|y))) for x,y in zip(df["antecedents"], df["consequents"])]
  df.drop_duplicates(subset=['association'], inplace=True)
  df.drop(columns=['association'], inplace=True)
  df.index=range(0, len(df))
  return df

In [ ]:
associations = drop_duplicate_association(ar).loc[:,["antecedents", "consequents", "antecedent support", "consequent support", "support", "confidence", "confidence_inverse", "conviction", "lift", "chi_square", "p_value", "leverage", "all_confidence", "max_confidence", "kulczynski", "cosine", "imbalance_ratio"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors

In [ ]:
association_rules

<function mlxtend.frequent_patterns.association_rules.association_rules(df, metric='confidence', min_threshold=0.8, support_only=False)>